In [3]:
import numpy as np
from geo3d.constraints import constrained_movement_2D, constrained_movement_3D
from geo3d.units import arcsec_in_degrees

## constrained movement in 2D
z is restrained to z=0.
Three constraints can exactly constrain a rigid body in 2D.

In [12]:
rs = [[0,1,0], [1,1,0], [2,0,0]] # constraint positions
cs = [[0,-1,0], [0,-1,0], [-1,0,0]] # constraint surface normal vectors
ds = [0,0.2,0] # movement deltas of the constraints along the surface normals 

In [13]:
constrained_movement_2D(rs, cs, ds)

## constrained movement in 3D

6 constraints can exactly constrain a rigid body in 3D.

In [19]:
rs = [[-65.25,  -9.33, -38.25],
      [ 54.25, -24.83, -38.25],
      [-54.25, -24.83, -38.25],
      [-54.25, -16.83, -37.00],
      [ 54.25, -16.83, -37.00],
      [  0.00,  61.17, -37.00]
     ]  # constraint positions
cs = [
    [1,0,0], 
    [0,1,0], 
    [0,1,0], 
    [0,0,1], 
    [0,0,1], 
    [0,0,1]
    ] # constraint surface normal vectors
ds = [0,0,0,.25,.25,.25] # movement deltas of the constraints along the surface normals 

In [20]:
constrained_movement_3D(rs, cs, ds)

## Uncertainty propagation for 3D excactly constrained movement

In [31]:
rs = [[-65.25,  -9.33, -38.25],
      [ 54.25, -24.83, -38.25],
      [-54.25, -24.83, -38.25],
      [-54.25, -16.83, -37.00],
      [ 54.25, -16.83, -37.00],
      [  0.00,  61.17, -37.00]
     ]
cs = [
    [1,0,0], 
    [0,1,0], 
    [0,1,0], 
    [0,0,1], 
    [0,0,1], 
    [0,0,1]
    ]

## Propagation constraint accuracies

Uncertainties of the constraints along the surface normals propagated to a total rigid body positioning uncertainty. 

### Uncorrelated uncertainties

Consider uncertainties along the different constraints are uncorrelated. 
Then, take square root of the sum of squares of the the rigid-body movement parameters. 

In [32]:
IP_acc = 27.6e-3
OOP_acc = 17.9e-3
ds_acc = [IP_acc,IP_acc,IP_acc,OOP_acc,OOP_acc,OOP_acc]

ps = []
for i,d in enumerate(ds_acc):
    ds = np.zeros(6)
    ds[i] = d
    t = constrained_movement_3D(rs, cs, ds)
    p = np.empty(6)
    p[0:3] = t.euler_angles('xyz', degrees=True)/arcsec_in_degrees
    p[3:]  = np.array(t.translation)*1000
    ps.append(p)
ps = np.array(ps)
acc = np.sqrt(np.sum(ps**2,0))
print(acc)

[57.97345551 48.12418794 74.20265875 29.20161112 22.28203187 10.64967756]


### Stochastic propagation of constraint accuracies

Stochastic simulation of Gaussian distribution of constraint accuracies.

In [40]:
# stochastic uncertainty propgation
ps = []
for i in range(100):
    ds = np.random.normal(scale=ds_acc)
    t = constrained_movement_3D(rs, cs, ds)
    p = np.empty(6)
    p[0:3] = t.euler_angles('xyz', degrees=True)/arcsec_in_degrees
    p[3:]  = np.array(t.translation)*1000
    ps.append(p)
ps = np.array(ps)
acc = np.std(ps,0)
print(acc)

[55.09574244 44.24180607 73.66732076 29.76089936 22.11755524 11.22613956]
